<h1>Example Langchain with MSSQL Server</h1>

<h4>Initialize the DB</h4>

In [ ]:
!pip install pyodbc

In [1]:
# from langchain.sql_database import SQLDatabase
from Custom_SqlDatabase import SQLDatabase

# Replace the following with your actual connection details
username = 'user1'
password = 'IoP3X5_QRFiE'
servername = 'localhost'  # or your server's name
databasename = 'FoodFare'

# MSSQL connection URI
connection_uri = f"mssql+pyodbc://{username}:{password}@{servername}/{databasename}?driver=ODBC+Driver+17+for+SQL+Server"
#connection_uri = f"mssql+pyodbc://{username}:{password}@{servername}/{databasename}?driver=SQL+Server"

db = SQLDatabase.from_uri(connection_uri, schemas=['dbo','Sales'])
print(db)

j:\AI\LLM-Course-2\LangChain-101-For-Beginners-Python\Custom_SqlDatabase.py:93: SAWarning: Did not recognize type 'geography' of column 'SpatialLocation'
  self._metadata.reflect(
j:\AI\LLM-Course-2\LangChain-101-For-Beginners-Python\Custom_SqlDatabase.py:93: SAWarning: Did not recognize type 'hierarchyid' of column 'OrganizationNode'
  self._metadata.reflect(


In [2]:
from langchain_community.agent_toolkits import (create_sql_agent, SQLDatabaseToolkit)
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.sql.prompt import SQL_FUNCTIONS_SUFFIX
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

toolkit = SQLDatabaseToolkit(db=db, llm=ChatOpenAI(temperature=0))
tools = toolkit.get_tools()

messages = [
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessage(content=SQL_FUNCTIONS_SUFFIX),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
]

prompt = ChatPromptTemplate.from_messages(messages)



In [3]:
from langchain.agents import create_openai_tools_agent
from langchain.agents.agent import AgentExecutor

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=True,
)

In [53]:
agent_executor.invoke({"input": "Describe the schema of the Customer table"})



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


Sales.CountryRegionCurrency, Sales.CreditCard, Sales.Currency, Sales.CurrencyRate, Sales.Customer, Sales.PersonCreditCard, Sales.SalesOrderDetail, Sales.SalesOrderHeader, Sales.SalesOrderHeaderSalesReason, Sales.SalesPerson, Sales.SalesPersonQuotaHistory, Sales.SalesReason, Sales.SalesTaxRate, Sales.SalesTerritory, Sales.SalesTerritoryHistory, Sales.ShoppingCartItem, Sales.SpecialOffer, Sales.SpecialOfferProduct, Sales.Store, dbo.AWBuildVersion, dbo.DatabaseLog, dbo.ErrorLog, dbo.Test
Invoking: `sql_db_schema` with `{'table_names': 'Sales.Customer'}`



CREATE TABLE [Sales].[Customer] (
	[CustomerID] INTEGER NOT NULL IDENTITY(1,1), 
	[PersonID] INTEGER NULL, 
	[StoreID] INTEGER NULL, 
	[TerritoryID] INTEGER NULL, 
	[AccountNumber] AS ((isnull('AW'+[dbo].[ufnLeadingZeros]([CustomerID]),''))) NOT NULL, 
	rowguid UNIQUEIDENTIFIER NOT NULL DEFAULT (newid()), 
	[ModifiedDate] DATETIME NOT NUL

{'input': 'Describe the schema of the Customer table',
 'output': "The schema of the Customer table is as follows:\n\n- CustomerID: INTEGER, NOT NULL, IDENTITY(1,1)\n- PersonID: INTEGER, NULL\n- StoreID: INTEGER, NULL\n- TerritoryID: INTEGER, NULL\n- AccountNumber: Computed column (AS ((isnull('AW'+[dbo].[ufnLeadingZeros]([CustomerID]),''))), NOT NULL\n- rowguid: UNIQUEIDENTIFIER, NOT NULL, DEFAULT (newid())\n- ModifiedDate: DATETIME, NOT NULL, DEFAULT (getdate())\n\nPrimary Key:\n- PK_Customer_CustomerID: CustomerID\n\nForeign Keys:\n- FK_Customer_Person_PersonID: PersonID references Person.BusinessEntityID\n- FK_Customer_SalesTerritory_TerritoryID: TerritoryID references Sales.SalesTerritory.TerritoryID\n- FK_Customer_Store_StoreID: StoreID references Sales.Store.BusinessEntityID\n\nSample rows from the Customer table:\n| CustomerID | PersonID | StoreID | TerritoryID | AccountNumber | rowguid                              | ModifiedDate           |\n|------------|----------|---------|